In [1]:
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from pathlib import Path

def img_set(images_path):
    categories = ['bulbasaur', 'charmander', 'squirtle']
    images = []
    labels = []
    images_path = Path(images_path)
    for img in images_path.iterdir():
        image = load_img(img)
        image = img_to_array(image)
        label = categories.index(img.name.split('_')[0])
        labels.append(label)
        images.append(image)
    return np.array(images), np.array(labels)

In [2]:
train_dir = "../data/generated/train"
test_dir = "../data/generated/test"

x_train, y_train = img_set(train_dir)
x_test, y_test = img_set(test_dir)

In [4]:
y_test

array([0, 0, 0, ..., 1, 0, 0])

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.keras as keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D, Dropout

X_train = np.array(x_train).astype('float32')
X_test = np.array(x_test).astype('float32')
X_train /= 255
X_test /= 255

num_classes=3
Y_train = to_categorical(y_train, num_classes=num_classes)
Y_test = to_categorical(y_test, num_classes=num_classes)




In [6]:

model = Sequential()

model.add(Conv2D(32, (3,3), input_shape=X_train.shape[1:]))
model.add(Activation('relu'))

model.add(Conv2D(32, (3,3)))
model.add(Activation('relu'))

model.add(Conv2D(32, (3,3)))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3,3)))
model.add(Activation('relu'))

model.add(Conv2D(64, (3,3)))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(num_classes, input_shape=[num_classes]))
model.add(Activation('softmax'))

optimizer = optimizers.Adam()
model.compile(loss='categorical_crossentropy',
             optimizer=optimizer,
             metrics=['accuracy'])

epochs = 10
batch_size = 150


In [7]:

model.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, Y_test))

Epoch 1/10
517/517 [==============================] - 122s 236ms/step - loss: 0.2551 - accuracy: 0.8923 - val_loss: 0.0732 - val_accuracy: 0.9629
Epoch 2/10
517/517 [==============================] - 121s 234ms/step - loss: 0.0915 - accuracy: 0.9646 - val_loss: 0.0517 - val_accuracy: 0.9736
Epoch 3/10
517/517 [==============================] - 124s 239ms/step - loss: 0.0545 - accuracy: 0.9797 - val_loss: 0.0182 - val_accuracy: 0.9953
Epoch 4/10
517/517 [==============================] - 111s 214ms/step - loss: 0.0467 - accuracy: 0.9827 - val_loss: 0.0151 - val_accuracy: 0.9957
Epoch 5/10
517/517 [==============================] - 127s 245ms/step - loss: 0.0329 - accuracy: 0.9879 - val_loss: 0.0331 - val_accuracy: 0.9863
Epoch 6/10
517/517 [==============================] - 128s 247ms/step - loss: 0.0308 - accuracy: 0.9887 - val_loss: 0.0182 - val_accuracy: 0.9943
Epoch 7/10
517/517 [==============================] - 127s 245ms/step - loss: 0.0279 - accuracy: 0.9901 - val_loss: 0.0127 -

In [19]:
from PIL import Image
from pathlib import Path
def recognize_image(image_path):
    categories = ['bulbasaur', 'charmander', 'squirtle']
    image_path = Path(image_path)
    resized_image_path = image_path.parent/f"resized_{image_path.name}"
    img = Image.open(image_path)
    img = img.convert("RGB")
    img = img.resize((32,32))
    img.save(resized_image_path)
    
    image = load_img(resized_image_path)
    image = img_to_array(image)
    img_input_array = np.array([image])
    predictions = model.predict(img_input_array)
    prediction = np.argmax(predictions, axis=1)
    return categories[prediction[0]]

In [20]:
recognize_image("../charmander.jpg")

'charmander'

In [24]:
recognize_image("../test99.jpg")

'bulbasaur'

In [25]:
model.save_weights('../data/weights')